This is a data cleanup/merge notebook. 

In [1]:
#Import libraries
import pandas as pd
import numpy as np

In [15]:
#Load Data Frames
Unemployment_data = pd.read_csv("unemployment.csv")
financial_data = pd.read_csv("Financial Data.csv")
consumption_data = pd.read_csv("coffee_consumption.csv", header=3, encoding="latin")
gdp_perhw_data = pd.read_csv("gdp_hrwkd.csv")
srini_data = pd.read_csv("Coffee_Consuption_clean.csv")
consumption_data.head

<bound method NDFrame.head of                                 Austria   1,340   1,302   1,212   1,335  \
0                               Belgium     NaN     NaN     NaN     NaN   
1                    Belgium/Luxembourg   1,124     708     696     789   
2                              Bulgaria     102      17     175     327   
3                               Croatia     NaN     NaN     135     132   
4                                Cyprus      31      45      39      78   
5                        Czech Republic     659     525     472     448   
6                               Denmark     865     909     959     891   
7                               Estonia     NaN     NaN      13      44   
8                               Finland   1,070     966   1,030   1,129   
9                                France   5,205   5,557   5,614   5,507   
10                              Germany   7,279  12,377   9,570  11,325   
11                               Greece     631     387     320     14

In [16]:
#Remove N/A rows from Consumption Data Frame

consumption_clean_rows = consumption_data.dropna(thresh=11)

#Filter for data for 1999-2013
consumption_clean = consumption_clean_rows.dropna(axis=1)
consumption_clean.head()


,Austria,"1,123",875,"1,013",926,720,996,772,612,847,908,886,903,"1,117","1,269","1,249"
0,Belgium,834,"1,133",884,"1,484","1,579","1,396","1,158","1,537","1,103",650,934,871,934,915,"1,245"
2,Bulgaria,363,275,350,341,397,364,430,420,364,445,409,395,360,376,419
3,Croatia,337,327,337,355,375,370,374,386,386,377,366,370,367,360,387
4,Cyprus,54,67,53,51,53,60,70,55,69,77,75,74,81,85,89
5,Czech Republic,558,560,633,642,623,605,656,631,679,621,525,470,572,685,637


In [17]:
#Rename Calendar years as Country Name
consumption_clean.rename(index=str, columns={"Calendar years": "Country Name"}, inplace=True)
consumption_clean.columns


/anaconda2/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


Index(['   Austria', '1,123', '875', '1,013', '926', '720', '996', '772',
       '612', '847', '908', '886', '903', '1,117', '1,269', '1,249'],
      dtype='object')

In [7]:
#Move Years to a single column
consumption_clean2 = pd.melt(consumption_clean, id_vars=['Country Name'], var_name="Year", value_name="Coffee Consumed")
consumption_clean2



KeyError: 'Country Name'

In [ ]:
#Remove Extra spaces in front of Country Names
consumption_clean2["Country Name"] = consumption_clean2["Country Name"].str.strip()


In [ ]:
#View Financial Data Set
financial_data.head()


In [ ]:
#Drop Unneeded Columns
financial_data_clean = financial_data.drop(columns=['sex', 'age', 'suicides_no',
                                                    'suicides/100k pop', 'country-year', 'HDI for year', 'generation'])
financial_data_clean.columns


In [ ]:
#Group by Country and Year
financial_group = financial_data_clean.groupby(["country", "year", " gdp_for_year ($) ",
                                                "gdp_per_capita ($)"])

total_pop = financial_group.sum()
total_pop.reset_index(inplace=True)
total_pop.head()

In [ ]:
#rename country to Country Name and year to Year
total_pop.rename(index=str, columns={"country": "Country Name","year": "Year" }, inplace=True)


In [ ]:
total_pop.head()

In [ ]:
#View GDP per Hour Worked file
gdp_perhw_data.dtypes


In [ ]:
#Drop Unneeded Columns
gdph_clean = gdp_perhw_data.drop(columns=['LOCATION', 'INDICATOR', 'SUBJECT',
                                                    'FREQUENCY', 'Flag Codes'])
#rename Country to Country Name
gdph_clean.rename(index=str, columns={"Country": "Country Name", "TIME": "Year"}, inplace=True)
gdph_clean.head()

In [ ]:
#Remove extra spaces in front of country name
gdph_clean["Country Name"] = gdph_clean["Country Name"].str.strip()

#Convert Year to object
gdph_clean["Year"] = gdph_clean["Year"].astype(object)



In [ ]:
#View Unemployment data
Unemployment_data.head()

In [ ]:
#Drop Uneeded Columns
Unemployment_data.drop(columns=['Country Code', 'Indicator Name'], inplace = True)
Unemployment_data.head()

In [ ]:
#Rename Value to %Unemployed
Unemployment_data.rename(index=str, columns={"Value": "% Unemployed"}, inplace=True)
Unemployment_data.head()

In [ ]:
consumption_clean2['Country Name'] = consumption_clean2['Country Name'].astype(str)
consumption_clean2['Year'] = consumption_clean2['Year'].astype(str)
consumption_clean2.dtypes

In [ ]:
gdph_clean['Country Name'] = gdph_clean['Country Name'].astype(str)
gdph_clean['Year'] = gdph_clean['Year'].astype(str)
gdph_clean.dtypes

In [ ]:
total_pop['Year'] = total_pop['Year'].astype(str)
total_pop.dtypes

In [ ]:
total_pop['Year'] = total_pop['Year'].astype(str)
Unemployment_data.dtypes

In [ ]:
#Merge Data Sets 
combine_1 = pd.merge(consumption_clean2, gdph_clean, how='left', on=['Country Name', 'Year'])
combine_1.head()


In [ ]:
combine_2 = pd.merge(combine_1, total_pop, how='left', on=['Country Name', 'Year'])
combine_2.head()

In [ ]:
combine_3 = pd.merge(combine_2, Unemployment_data, how='left', on=['Country Name', 'Year'])
combine_3.head()

In [ ]:
combine_clean = combine_3.loc[combine_3["MEASURE"] == "USD", :]
combine_clean.head()

In [ ]:
combine_clean.count()

In [ ]:
#Drop NA's
combine_clean2 = combine_clean.dropna(how='any')
combine_clean2.count()

In [ ]:
#Write to csv
combine_clean2.to_csv("project data.csv", index=False)

In [ ]:
srini_data.head()
srini_data.rename(index=str, columns={"Value": "Coffee Consumed"}, inplace=True)

srini_data.head()

In [ ]:
srini_data['Year'] = srini_data['Year'].astype(str)
srini_data['Country Name'] = srini_data['Country Name'].astype(str)
srini_data.dtypes

In [ ]:
scombine_1 = pd.merge(srini_data, gdph_clean, how='left', on=['Country Name', 'Year'])
scombine_1.head()

In [ ]:
scombine_2 = pd.merge(scombine_1, total_pop, how='left', on=['Country Name', 'Year'])
scombine_2.head()

In [ ]:
scombine_3 = pd.merge(scombine_2, Unemployment_data, how='left', on=['Country Name', 'Year'])
scombine_3.head()

In [ ]:
scombine_clean = scombine_3.loc[scombine_3["MEASURE"] == "USD", :]

In [ ]:
#Drop NA's
scombine_clean2 = scombine_clean.dropna(how='any')
scombine_clean2.count()

In [ ]:
scombine_clean2.to_csv("project data(srini data).csv", index=False)